In [1]:
%run ../../global-tools/start.py

# make sure the figures plot inline rather than at the end
%matplotlib inline

Default Libraries Loaded.


# Seasonal Ekman Pumping

To isolate interior ekman pumping, this code follows https://agupubs.onlinelibrary.wiley.com/doi/epdf/10.1029/2011JC007531

Need to still do alongshore wind stress at some point too...need local bathymetry for that...perhaps cartopy has it.

https://earthweb.ess.washington.edu/roe/Web_page_514/ESS514/Notes_files/MarshallPlumb_GulfStream.pdf

In [69]:
ds_wnd = xr.open_dataset('../data/NCEP_Wind/ncep_wind_processed.nc')

# Subset ------------------------------------------------#
# Create slice variables to subset domain before finding means
lat_slice = slice(-20, 30) # bounds inclusive
lon_slice = slice(35, 120) # bounds inclusive
time_slice = slice('1994-01-01','2019-5-31')

# Get data, selecting lat/lon slice
u = ds_wnd['uwnd'].sel(lat=lat_slice,lon=lon_slice, time = time_slice)
v = ds_wnd['vwnd'].sel(lat=lat_slice,lon=lon_slice, time = time_slice)
lat = u.lat.values
lon = u.lon.values

In [72]:
# wind stress

# air density
p_air = 1.225 # kg/m^3
p_sea = 1025 # kg/m^3

xx,yy = np.meshgrid(ds_wnd.lon,ds_wnd.lat)

n = len(ds_wnd.time)
m = len(ds_wnd.lat)
p = len(ds_wnd.lon)

dx,dy = mpcalc.lat_lon_grid_deltas(xx,yy)

In [ ]:
C_d = np.full(size(tmplon),np.nan)

# n = len(ds_wnd.time)
# m = len(ds_wnd.lat)
# p = len(ds_wnd.lon)

# with tqdm(total=n * m * p) as pbar:
#     for tt,ii,jj in itertools.product(np.arange(n),np.arange(m),np.arange(p)):
for tt in tqdm(range(len(ds_wnd.time))):
    tmpu = ds_wnd.uwnd[tt,:,:]
    tmpv = ds_wnd.vwnd[tt,:,:]

    # define wind speed
    W = np.sqrt(tmpu.^2 + tmpv.^2)

    # define drag coeficient matrix 
    C_d[W < 11] = 0.0012
    C_d[W >= 11] = 0.00049 + U*0.000065

    # calculate wind stress
    tau_x = p_air * C_D * W * tmpu
    tau_y = p_air * C_D * W * tmpv

    # find wind stress gradients
    dtau_x_dx, dtau_x_dy = np.gradient(tau_x, dx,dy)
    dtau_y_dx, dtau_y_dy = np.gradient(tau_y, dx,dy)

    # find wind stress curl
    curl_tau = dtau_y_dx - dtau_x_dy

    f = mpcalc.coriolis_parameter(tmplat)

    curl_tau = 

    W_Ek = curl_tau./(p_sea * f)

        
#         pbar.update(1)

In [ ]:
def latlon_to_xyz(lat,lon):
    """Convert angluar to cartesian coordiantes

    latitude is the 90deg - zenith angle in range [-90;90]
    lonitude is the azimuthal angle in range [-180;180] 
    """
    r = 6371 # https://en.wikipedia.org/wiki/Earth_radius
    theta = math.pi/2 - math.radians(lat) 
    phi = math.radians(lon)
    x = r * math.sin(theta) * math.cos(phi) # bronstein (3.381a)
    y = r * math.sin(theta) * math.sin(phi)
    z = r * math.cos(theta)
    return [x,y,z]